## 5層CNN(Convolutional Neural Network)と3層NN(Neural Network)の精度比較

本プログラムの実行によって同ディレクリ上に「result」フォルダが作成され，グラフが保存される．

また，CUDAが利用可能な場合，本プログラムの学習はGPUで計算される．動作確認した環境では実行後もGPUのメモリは開放されなかった．このような問題が生じた場合，プログラム実行後にツールバー>File>Close and Halt 等で本プログラムを終了することでメモリが開放される．

In [ ]:
import torch # torchモジュール のインポート
import torchvision # torchvisionモジュール のインポート
import torch.nn as nn # torchのニューラルネットモジュールをnnと名付けインポート
import torch.nn.functional as F # torchのニューラルネット関数をFと名付けてインポート
import torch.optim as optim # torchの最適化モジュールをoptimと名付けてインポート
from torchvision import datasets, transforms
import numpy as np
import matplotlib.pyplot as plt
import os

In [ ]:
# Graph描画クラス
class Graph:
    def __init__(self):
        # ----ユーザー設定値----
        self.count = 1  # 保存画像のナンバリング初期値
        self.fontsize = "small"  # 散布図のテキストのフォントサイズの設定   x0.833 相対的なサイズ
        self.markers = [".", "v", "x", "1", ","]  # 近似手法の数だけ，マーカーを設定
        self.lineStyle = ["solid", "dashed", "dotted", "solid", "dashed", "dotted"]
        self.saveDirectory = "result/"  # 保存するディレクトリを指定
        if not os.path.exists(self.saveDirectory):  # ディレクトリが存在しない場合
            os.makedirs(self.saveDirectory)  # ディレクトリを作成

    # 散布図作成・保存メソッド(marker only)
    def scatterDraw(self, scatterLabels, figName, logFlag, *args):  #
        # scatterLabels:グラフタイトル,x軸タイトル、y軸タイトル, figName：参照メソッドに関連する名前, logFlag:対数グラフ化のフラグ
        # args:(各次元のlogデータ,timeデータ,手法名)のデータセット

        # ---グラフ設定----
        fig, ax = plt.subplots(1, 1, figsize=(12, 9))  # 拡張性確保のためにsubplotで生成．
        plt.subplots_adjust(bottom=0.15)  # グラフの下部空白を拡張

        # ---対数グラフ化----
        if logFlag:
            ax.set_yscale('log')
            ax.set_xscale('log')

        # ----グラフ作成-----
        # グラフに写像するデータの種類分だけループ
        for i in range(len(args)):
            ax.scatter(args[i][0], args[i][1], label=args[i][2], marker=self.markers[i])  # 散布図を作成
        ax.set_title(scatterLabels[0], y=-.2)  # タイトル設定、枠外下部に来るように調整
        ax.set_xlabel(scatterLabels[1])  # x軸ラベルの設定、枠外下部に来るように調整
        ax.set_ylabel(scatterLabels[2])  # y軸ラベルの設定、枠外左部に来るように調整
        plt.grid()  # グリッドを追加
#        ax.legend(loc='lower right')  # 凡例の位置を設定
        # plt.show()

        # ---グラフの保存---
        fig.savefig(f"{self.saveDirectory}scatter{self.count}_{figName}.png")  # 画像の保存
        plt.close()  # 大量画像データ処理によるメモリ不足エラーを避ける．
        self.count += 1  # 保存用ナンバーの更新
        
        # 散布図作成・保存メソッド(line)
    def plotGraph(self, scatterLabels, figName, logFlag, *args):  #
        # scatterLabels:グラフタイトル,x軸タイトル、y軸タイトル, figName：参照メソッドに関連する名前, logFlag:対数グラフ化のフラグ
        # args:(各次元のlogデータ,timeデータ,手法名)のデータセット

        # ---グラフ設定----
        fig, ax = plt.subplots(1, 1, figsize=(12, 9))  # 拡張性確保のためにsubplotで生成．
        plt.subplots_adjust(bottom=0.15)  # グラフの下部空白を拡張

        # ---対数グラフ化----
        if logFlag:
#            ax.set_yscale('log')
            ax.set_xscale('log')

        # ----グラフ作成-----
        # グラフに写像するデータの種類分だけループ
        for i in range(len(args)):
            ax.plot(args[i][0], args[i][1], label=args[i][2], linestyle=self.lineStyle[i])  # 散布図を作成
        ax.set_title(scatterLabels[0], y=-.2)  # タイトル設定、枠外下部に来るように調整
        ax.set_xlabel(scatterLabels[1])  # x軸ラベルの設定、枠外下部に来るように調整
        ax.set_ylabel(scatterLabels[2])  # y軸ラベルの設定、枠外左部に来るように調整
        plt.grid()  # グリッドを追加
        ax.legend(loc='lower right')  # 凡例の位置を設定
        # plt.show()

        # ---グラフの保存---
        fig.savefig(f"{self.saveDirectory}scatter{self.count}_{figName}.png")  # 画像の保存
        plt.close()  # 大量画像データ処理によるメモリ不足エラーを避ける．
        self.count += 1  # 保存用ナンバーの更新

In [ ]:
def divPrint(*args):  # 任意の数の変数を開業しながら表示。開始と終了を線で区切る。
    print("===========================================================\n")
    for i in args:
        print(f"{i}\n")
    print("\n===========================================================\n")


In [ ]:
class MyNN(nn.Module): # 自分のNNモデルを nn.Moduleのサブクラスとして定義
    def __init__(self, hiddenLayerNulonNum=200): # コンストラクタ
        super(MyNN, self).__init__() # 上位クラスのコンストラクタを継承
        self.fc1 = nn.Linear(in_features=28*28,out_features=hiddenLayerNulonNum) # 全結合 28*28→200
        self.fc2 = nn.Linear(in_features=hiddenLayerNulonNum,out_features=10)    # 全結合 200→10(0～9の十種類)
        self.dropout = nn.Dropout()
        self.bnorm = nn.BatchNorm1d(200)
    
    def forward(self,x): # 入力に対する結果を返すNNの定義。backwardは自動的に定義される
        # xのデータ構造を (バッチサイズ, 1, 28, 28) → (バッチサイズ, 28*28)に変換
        x = x.view(-1,28*28)
        # 入力層→中間層
        x = self.fc1(x)
        # 活性化関数 relu(x)=max(0,x)
        # x = F.relu(x)CrossEntropyLoss
        # 活性化関数 シグモイド
        x = torch.sigmoid(x)
        # 中間層→出力層
        output = self.fc2(x)
        #output = F.softmax(output, dim=1) # 通常はsoftmaxが必要だが、PyTorchではCrossEntropyにsoftmaxが入ってるので不要
        
        return output
    
class MyCNN(nn.Module): # 5層CNNのクラス
    def __init__(self): # コンストラクタ
        super(MyCNN,self).__init__() # 上位クラスのコンストラクタを継承
        self.firstSize = 128
        self.sizes =[self.firstSize, self.firstSize*2, self.firstSize*8]
        self.conv1 = nn.Conv2d(1, self.sizes[0], 5) # 28x28x1 -> 24x24x256 (kernel:5)
        self.pool = nn.MaxPool2d(2, 2) # 24x24x256 -> 12x12x256, 8x8x512 -> 4x4x512
        self.conv2 = nn.Conv2d(self.sizes[0], self.sizes[1], 5) # 12x12x256 -> 8x8x512 (kernel:5)
        self.fc1 = nn.Linear(in_features=4*4*self.sizes[1],out_features=self.sizes[2]) # 全結合 4x4x512→2048
        self.fc2 = nn.Linear(in_features=self.sizes[2],out_features=10)    # 全結合 2048→10(0～9の十種類)
        
    
    def forward(self,x): # 入力に対する結果を返すNNの定義。backwardは自動的に定義される
        # 入力層→中間層1
        x = self.pool(self.conv1(x))
        x = torch.relu(x)
        # 中間層1→中間層2
        x = self.pool(self.conv2(x))
        x = torch.relu(x)
        # 中間層2→中間層3
        # xのデータ構造を (バッチサイズ, 4, 4, 64) → (バッチサイズ, 4x4x64)に変換
        x = x.view(-1,4*4*self.sizes[1])
        x = self.fc1(x)
        x = torch.relu(x)
        # 中間層3→出力層
        output = self.fc2(x)
        #output = F.softmax(output, dim=1) # 通常はsoftmaxが必要だが、PyTorchではCrossEntropyにsoftmaxが入ってるので不要
        
        return output

In [ ]:
def train(model, loader, objective, optimizer, epoch, device):
    # 
    model.train()
    total_loss = 0.0
    for count, (data, target) in enumerate(loader):
        
        data = data.to(device) # GPUを使用するため，to()で明示的に指定
        target = target.to(device) # 同上
#        print(data.device, target.device)
        
        optimizer.zero_grad()
        output = model(data)
        loss = objective(output,target) # 損失(CrossEntropy)を計算     
        total_loss += loss
        
        loss.backward() # 誤差逆伝播
        optimizer.step()
        if ((count+1)%100==0):
            print("[%d] loss=%g"%((count+1),loss))
    print("Average loss = %g"%(total_loss/(count+1)) )
    
def test(model, loader, objective, device):
    model.eval()
    total = 0
    total_loss = 0
    correct = 0
        
    with torch.no_grad():
        for count, (data, target) in enumerate(loader):
            
            data = data.to(device) # GPUを使用するため，to()で明示的に指定
            target = target.to(device) # 同上
            
            output = model(data)
            loss = objective(output,target) # 損失(交差エントロピー)を計算     
            total_loss += loss
            _, predicted = torch.max(output.data,1)
            correct += (predicted==target).sum().item() # sumの結果はテンソル
            total += target.size(0)
        accuracy = correct/total
    print("loss = %g, accuracy = %g"%(total_loss/(count+1),accuracy*100) )
    return accuracy*100

In [ ]:
def main():
    # GPUの設定（PyTorchでは明示的に指定する必要がある）
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print("device=", device)
    
    bsize=64

    # 学習データダウンロード
    trainval_set = datasets.MNIST('./data', train=True, download=True,
        transform=transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize((0.1307,), (0.3081,))
            ]))

    # テストデータダウンロード
    test_set = datasets.MNIST('./data', train=False, 
        transform=transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize((0.1307,), (0.3081,))
            ]))

    train_size = int(len(trainval_set)*0.9)
    validation_size = len(trainval_set)-train_size
    train_set, validation_set = torch.utils.data.random_split(trainval_set, [train_size,validation_size] )

    print("train_set: ", type(train_set), len(train_set))
    print("validation_set: ", type(validation_set), len(validation_set))
    print("test_set: ", type(test_set), len(test_set))

    train_loader = torch.utils.data.DataLoader(
        train_set, batch_size=bsize, shuffle=True )

    validation_loader = torch.utils.data.DataLoader(
        validation_set, batch_size=bsize, shuffle=True )

    test_loader = torch.utils.data.DataLoader(
        test_set, batch_size=bsize, shuffle=True )

    for batch_idx, (data, target) in enumerate(train_loader):
        print("Batch shape: ",data.size())
        break

    # 3層ニューラルネットのオブジェクトを生成
    setmodel = MyNN(hiddenLayerNulonNum=1000)
    mymodels = [setmodel.to(device)]
    # 5層畳み込みニューラルネットのオブジェクトを生成
    setmodel = MyCNN()
    mymodels.append(setmodel.to(device))
    print("Defined models=",mymodels)
    objective = nn.CrossEntropyLoss()
    
    # 5層CNNと3層NNの精度検証用の変数の定義        
    netName=["3layers-NN", "5layers-CNN"]
    dataSet=[]
    graph=Graph()
    maxEpoch=200
    divPrint("max Epoch",maxEpoch)
    lr=0.1  # 3層NNの学習率

    # mymodelをmymodelsの要素で設定し，学習を実施．
    for mymodel, network in zip(mymodels, netName):
        # 最適化手法いろいろ
        if network=="5layers-CNN":
            lr=0.001  # 5層CNNの学習率
        optimizer = optim.SGD(mymodel.parameters(), lr=lr, momentum=0.9)
    #     optimizerSet.append(optim.RMSprop(mymodel.parameters()))
    #     optimizerSet.append(optim.Adadelta(mymodel.parameters()))
    #     optimizerSet.append(optim.Adagrad(mymodel.parameters()))
    #     optimizerSet.append(optim.Adam(mymodel.parameters()))
    
        accuracySet = []
        epochSet = []
        divPrint("Network", network)
        for epoch in range(1, maxEpoch+1):
            print("--- Epoch %d ---"%epoch)
            train(mymodel, train_loader, objective, optimizer, epoch, device)
            print("Validation --> ", end="")
            test(mymodel, validation_loader, objective, device)
            print("Test --> ", end="")
            accuracy=test(mymodel, test_loader, objective, device)
            epochSet.append(epoch)
            accuracySet.append(accuracy)
        dataSet.append([epochSet, accuracySet, network])
        divPrint("Network", network,"epoch",epoch,"accuracy",accuracy)
        
    # グラフの出力        
    scatterLabels=["epoch-accuracy(Networks)", "epoch", "accuracy [%]"]
    divPrint(*dataSet)
    graph.plotGraph(scatterLabels, "Epoch-accuracy(Networks)", True, *dataSet)

In [ ]:
main()